In [1]:
%pylab osx

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import csv, os
import scipy.optimize
import scipy.special as ss
from math import *
π = np.pi

/usr/local/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [18]:
Res = pd.DataFrame()

for i in range(200):
    itempath = "langevin-reset-x0var/"+str(i+1)
    ppath = itempath+"-params.csv"
    if os.path.exists(ppath):
    #    res = {}
        df = pd.read_csv(ppath, sep=',', header=None)
        df = df.set_index(0).T
        df['i'] = i
        if not 'mfpt' in df.columns:
            first_times = np.array(pd.read_csv(itempath+"-ft.csv", names=['ft'])['ft'])
            df['mfpt'] = np.mean(first_times)
            df['fpt_stdev'] = np.std(first_times)
            df['n_traj'] = len(first_times)
            del first_times
        if not 'Delta_t' in df.columns:
            df['Delta_t'] = 50 * 1.5e-6
    #    for key in df:
    #        res[key] = df[key][1]
    #    Res.append(res)
        if (df['Delta_t']/df['part_m']).iloc[0] < 1e-3:
            Res = Res.append(df, ignore_index=True, sort=False)

len(Res)

122

In [19]:
Res['tau'] = 4*Res['D']*Res['mfpt']/Res['L']**2
Res['tau_err'] = 4*Res['D']*(Res['fpt_stdev']/np.sqrt(Res['n_traj']))/Res['L']**2

In [63]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(1,1,1, projection='3d')
ax.scatter( Res['b'], Res['c'], Res['tau'] )
plt.xlabel(r"$b=x_t/\sigma$")
plt.ylabel(r"$c=L\sqrt{\alpha/D}$")
#plt.zlabel(r"$\tau$")
plt.show()

In [20]:
plt.figure(figsize=(10,7))
groups = Res.groupby( Res['b'].apply(lambda x: round(x,2)) )
for b, Res_b in list(groups):
    if len(Res_b) > 4:
        c = np.linspace(0.4, 14, 200)
        tau = 4/c**2 * ( (2*np.exp(-c**2/2/b**2)) / ( np.exp(c)*ss.erfc((c/b+b)/sqrt(2)) + np.exp(-c)*ss.erfc((c/b-b)/sqrt(2)) ) - 1 )
        p = plt.plot(c, tau)
        c = np.array(Res_b['c'])
        i = np.argsort(c)
        c = c[i]
        tau = np.array(Res_b['tau'])[i]
        err = np.array(Res_b['tau_err'])[i]
        plt.errorbar(c, tau, label=r"$b={:.2f}$".format(b), color=p[0].get_color(), yerr=err, fmt='.')
        for i in range(len(Res_b)):
            res = Res_b.iloc[i]
            plt.annotate("$\\alpha={}$, $L={:.2f}$, $m={}$, $\Delta t$={:.1e}".format(res['reset_rate'],res['L'],res['part_m'],res['Delta_t']), (res['c'],res['tau']), size=6, ha='center', va='center')
            #plt.annotate(str(int(res['i'])), (res['c'],res['tau']), size=8, ha='center', va='center')
        
plt.xlabel(r"$c=L\sqrt{\alpha/D}$")
plt.ylabel(r"$\tau$")
plt.legend()
plt.show()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


In [22]:
plt.figure(figsize=(10,7))
groups = Res.groupby( Res['b'].apply(lambda x: round(x,2)) )
for b, Res_b in list(groups):
    if b > 1 and len(Res_b) > 3:
        c = np.linspace(0.4, 6, 200)
        tau = 4/c**2 * ( (2*np.exp(-c**2/2/b**2)) / ( np.exp(c)*ss.erfc((c/b+b)/sqrt(2)) + np.exp(-c)*ss.erfc((c/b-b)/sqrt(2)) ) - 1 )
        p = plt.plot(c, c*tau)
        c = np.array(Res_b['c'])
        i = np.argsort(c)
        c = c[i]
        tau = np.array(Res_b['tau'])[i]
        err = np.array(Res_b['tau_err'])[i]
        plt.errorbar(c, c*tau, label=r"$b={:.2f}$".format(b), color=p[0].get_color(), yerr=err, fmt='.')
#        for i in range(len(Res_b)):
#            res = Res_b.iloc[i]
#            plt.annotate("$\\alpha={}$\n$L={:.2f}$\n$D={}$".format(res['reset_rate'],res['L'],res['D']), (res['c'],res['c']*res['tau']), size=8, ha='center', va='center')
plt.xlabel(r"$c=L\sqrt{\alpha/D}$")
plt.yscale('log')
plt.xscale('log')
plt.ylabel(r"$c\cdot\tau$")
plt.legend()
plt.show()